In [ ]:
import pandas as pd
import numpy as np
from retrieve.get_data import get_yield
from statsmodels.tsa.api import VAR

# ----------------------------
# Load + prepare yield changes
# ----------------------------
df = get_yield(term=1)

# Weekly Friday close so all countries aligned on same calendar frequency
df = df.resample("W-FRI").last()

# Work in yield CHANGES (rates trading predicts moves, not levels)
df = df.diff().dropna()

# Cut sample to fixed end date (prevents future leakage when rerunning later)
df = df.truncate(after=pd.to_datetime("2023-02-10"))

# Ensure clean numeric matrix for VAR
df = df.sort_index()
df = df.apply(pd.to_numeric, errors="coerce").dropna()

# ----------------------------
# Strategy parameters
# ----------------------------
target = "UK"      # we trade UK rates
p = 5              # VAR lag length (chosen earlier via AIC)
k_vol = 0.5        # only trade if forecast > 0.5 * normal weekly move
vol_lookback = 104 # 2-year rolling volatility estimate
tcost = 0.0        # optional transaction cost penalty
vol_target = 0.0   # optional risk scaling

idx = df.index
y_uk = df[target].copy()

# Typical weekly UK move size — used as noise filter
rolling_vol = y_uk.rolling(vol_lookback).std()

# Storage for model forecast and trading position
pred = pd.Series(index=idx, dtype=float)
pos = pd.Series(index=idx, dtype=float)

# ----------------------------
# 70/30 train/test split
# ----------------------------
n = len(df)
split_i = int(np.floor(0.70 * n))
split_date = idx[split_i]

# We also need enough observations to estimate VAR and volatility
start_i = max(split_i, p + 1, vol_lookback + 1)

# ----------------------------
# Walk-forward backtest (true out-of-sample)
# Each week:
#   fit model using ONLY past data
#   forecast next week's UK yield change
#   decide position
# ----------------------------
for i in range(start_i, len(idx) - 1):
    end_date = idx[i]

    # expanding window: realistic learning through time
    train = df.loc[:end_date].copy()

    # Fit multi-country spillover model
    model = VAR(train)
    res = model.fit(p)

    # Predict next week changes using latest p observations
    last_lags = train.values[-p:]
    fcast = res.forecast(y=last_lags, steps=1)[0]

    # Extract UK prediction from vector
    uk_j = res.names.index(target)
    pred.loc[end_date] = fcast[uk_j]

    # --- Trading rule ---
    # Only trade if signal larger than normal noise
    vol = rolling_vol.loc[end_date]

    if pd.isna(vol) or vol == 0.0:
        pos.loc[end_date] = 0.0
    else:
        if abs(pred.loc[end_date]) < k_vol * vol:
            pos.loc[end_date] = 0.0
        else:
            # negative predicted Δy → yields down → bond price up → long duration
            pos.loc[end_date] = -np.sign(pred.loc[end_date])

    # Optional risk normalization
    if vol_target > 0:
        if (not pd.isna(vol)) and vol > 0:
            pos.loc[end_date] *= (vol_target / vol)
        else:
            pos.loc[end_date] = 0.0

pos = pos.fillna(0.0)

# ----------------------------
# PnL approximation
# price return ≈ -Δyield
# so long duration profits when yields fall
# ----------------------------
pnl = pos.shift(1) * (-y_uk)

# Turnover cost penalty
turnover = (pos - pos.shift(1)).abs().fillna(0.0)
pnl = pnl - tcost * turnover

# Evaluate ONLY true out-of-sample region
test_mask = idx >= split_date
pnl_bt = pnl.loc[test_mask].copy()

# ----------------------------
# Performance stats
# ----------------------------
cum = pnl_bt.cumsum()
mean = pnl_bt.mean()
std = pnl_bt.std(ddof=0)
sharpe = np.nan
if std and std > 0:
    sharpe = (mean / std) * np.sqrt(52)

trades = int((turnover.loc[pnl_bt.index] > 0).sum())
pct_in_market = (pos.loc[pnl_bt.index] != 0).mean()

print("Split date (70/30):", split_date)
print("Test window:", pnl_bt.index.min(), "->", pnl_bt.index.max())
print("Mean weekly PnL (per unit duration):", mean)
print("Std weekly PnL:", std)
print("Ann. Sharpe (52w):", sharpe)
print("Total cumulative PnL (per unit duration):", cum.iloc[-1])
print("Trades:", trades)
print("Pct weeks in market:", pct_in_market)

# Inspect last trades to sanity-check behavior
out = pd.DataFrame({
    "UK_dy": y_uk,
    "UK_pred_next": pred,
    "UK_vol": rolling_vol,
    "pos": pos,
    "pnl": pnl
})
print(out.loc[test_mask].tail(20))
